In [129]:
import pandas as pd
import numpy as np
import re
import gensim
from gensim.models import Word2Vec
from gensim.models import doc2vec
from scipy.spatial import distance
from nltk.corpus import stopwords
from gensim.models.doc2vec import LabeledSentence

In [4]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [19]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [86]:
comment_list=train['comment_text'].tolist()

# Analyse des données 

In [46]:
print('Nombre de commentaires:', train.shape[0])
print('Nombre de toxic:', train[train['toxic']==1].shape[0])
print('Nombre de severe_toxic:', train[train['severe_toxic']==1].shape[0])
print('Nombre de obscene:', train[train['obscene']==1].shape[0])
print('Nombre de threat:', train[train['threat']==1].shape[0])
print('Nombre de identity hate:', train[train['identity_hate']==1].shape[0])
print('Nombre de insult:', train[train['insult']==1].shape[0])

Nombre de commentaires: 159571
Nombre de toxic: 15294
Nombre de severe_toxic: 1595
Nombre de obscene: 8449
Nombre de threat: 478
Nombre de identity hate: 1405
Nombre de insult: 7877


In [138]:
train[train['insult']==1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
56,0020fd96ed3b8c8b,=Tony Sidaway is obviously a fistfuckee. He lo...,1,0,1,0,1,0
65,0028d62e8a5629aa,All of my edits are good. Cunts like you who ...,1,0,1,0,1,0
86,0036621e4c7e10b5,"Would you both shut up, you don't run wikipedi...",1,0,0,0,1,0
105,00472b8e2d38d1ea,A pair of jew-hating weiner nazi schmucks.,1,0,1,0,1,1
168,00686325bcc16080,"You should be fired, you're a moronic wimp who...",1,0,0,0,1,0
176,006b94add72ed61c,I think that your a Fagget get a oife and burn...,1,0,1,1,1,1


In [41]:
print('Nombre de commentaires considéré comme insulte non toxiques:',train[(train['toxic']==0)&(train['insult']==1)].shape[0])
print('On en déduit que tous les commentaires classé dans une catégories ne sont pas nécessairement toxiques ')

Nombre de commentaires considéré comme insulte non toxiques: 533
On en déduit que tous les commentaires classé dans une catégories ne sont pas nécessairement toxiques 


# Préparation des données

Avec les stopwords

In [118]:
def text_to_words(raw_text, remove_stopwords):
    # 1. Remove non-letters, but including numbers
    letters_only = re.sub("[^0-9a-zA-Z]", " ", raw_text)
    #letters_only = re.sub("[^0-9a-zA-Z]", " ", raw_text)
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english")) # In Python, searching a set is much faster than searching
        meaningful_words = [w for w in words if not w in stops] # Remove stop words
        words = meaningful_words
    return words 

sentences_train = train['comment_text'].apply(text_to_words, remove_stopwords=False)
sentences_test = test['comment_text'].apply(text_to_words, remove_stopwords=False)

Pour trainer le modèle je trouve les embeddings des phrases de test et de train

In [ ]:
sentences_train

# Creation des word embeddings

In [119]:
model = Word2Vec(sentences_train,size=100)

In [132]:
len(model.wv.vocab)

182267

In [115]:
model.most_similar('')

[('attucks', 0.6614641547203064),
 ('rutlish', 0.6503503918647766),
 ('nursery', 0.646953821182251),
 ('elementary', 0.6452115774154663),
 ('gag01001', 0.637749433517456),
 ('ateneo', 0.6299043893814087),
 ('crispus', 0.6208091974258423),
 ('pittsford', 0.6174736022949219),
 ('caddo', 0.6166243553161621),
 ('prebendal', 0.6160601377487183)]

In [71]:
Mat_ref= np.zeros((len(sentences_train),100),dtype="float32")
for i in range(len(sentences_train)):
    Mat_ref[i,]=model.wv[sentences_train[i]].sum(0)/len(sentences_train[i])

In [93]:
##a creuser

def most_similar_comment(comment):
    lista=[]
    for i in range(comment_list):
        b= np.array(Mat_ref[i])
        a= Mat_ref[comment]
        dst=distance.cosine(a, b)
        print(dst)
        lista.append(dst)
#on prend la deuxieme valeur minimale
    u=lista.index(sorted(lista)[1])
    v=comment_list[u]
    print(v)

In [94]:
np.array(Mat_ref[i])

array([ 0.07535175, -0.85153466,  0.12985575, -0.3485038 ,  0.51634884,
       -0.17087087,  1.9744736 ,  0.91666716,  0.22102638, -1.27888274,
        0.3578459 ,  0.29001927, -0.80352706,  0.55825502, -0.9047718 ,
        1.56521225, -0.00454149,  0.18085954,  1.40038097, -0.2569921 ,
       -1.78010082, -0.07857239, -0.92071813,  0.19051269, -0.62165862,
       -0.45281696, -1.17870092, -1.0245986 ,  0.08645821, -0.76097155,
       -0.27727783, -0.65798748,  0.35650185,  0.1941746 , -0.47701931,
        0.1618941 ,  0.36496913, -0.39012837, -0.94928765,  1.02819788,
       -0.29291013, -0.3047083 ,  0.41594252, -0.8088001 ,  0.93490124,
       -0.87048751,  0.92747247, -0.2074167 ,  0.12084217, -0.08398182,
       -0.11064754, -1.49293411, -0.50134403, -0.87234056,  0.21378523,
        0.49643192, -0.33052558,  0.1858481 , -0.88144088,  0.38988125,
        0.05891834, -1.04628146,  0.26619589,  0.64142299, -0.51320249,
       -0.82635808,  0.06913158, -0.68545991, -0.18035935,  0.87

In [164]:
labeled_comments=[]
for i in range(len(sentences_list)):
    labeled_comments.append(LabeledSentence(words=sentences_list[i], tags=[id_list[i]]))

In [125]:
sentences_list=sentences_train.tolist()
id_list=train['id'].tolist()

In [136]:
model_d2v = gensim.models.Doc2Vec(labeled_comments)

In [159]:
def most_sim_d2v(comment_id):
    print('Comment:',train[train['id']==comment_id]['comment_text'])
    most_sim=model_d2v.docvecs.most_similar(comment_id,topn=10)
    print('10 most similar comments:')
    for i in range(len(most_sim)):
        print(train[train['id']==most_sim[i][0]]['comment_text'])
    

In [162]:
model_d2v.docvecs.most_similar('ff39a2895fc3b40e',topn=10)

[('9f1956011b8f3813', 0.9043959379196167),
 ('de2fda24221bf8e3', 0.9035018682479858),
 ('403e1ad458d28fb9', 0.900415301322937),
 ('fcb14ba4a8a9ce3e', 0.8997960090637207),
 ('e3a130469e30aa44', 0.8978034257888794),
 ('0232e2de22311164', 0.8975162506103516),
 ('3ff5f55c0f083853', 0.8974905014038086),
 ('6df36bf15864c4b6', 0.897163987159729),
 ('093a31ee04c6565f', 0.8959699273109436),
 ('199ec3f36ccbd9d3', 0.8953325748443604)]

In [163]:
train[train['id']=='9f1956011b8f3813']

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
125458,9f1956011b8f3813,"""\n\nFair use rationale for Image:Chamillionai...",0,0,0,0,0,0


In [161]:
most_sim_d2v('ff39a2895fc3b40e')

Comment: 159514    YOU ARE A MISCHIEVIOUS PUBIC HAIR
Name: comment_text, dtype: object
10 most similar comments:
125458    "\n\nFair use rationale for Image:Chamillionai...
Name: comment_text, dtype: object
157439    REDIRECT Talk:Hannah Hodson (actress)
Name: comment_text, dtype: object
24330    January 2015 \nhttp://en.wikipedia.org/wiki/Al...
Name: comment_text, dtype: object
142999    Welcome!\n\nHello, , and welcome to Wikipedia!...
Name: comment_text, dtype: object
85071    Hello, , and welcome to Wikipedia! Thank you f...
Name: comment_text, dtype: object
823    Barnes                  Aus     1             ...
Name: comment_text, dtype: object
147667    "\n\nSpeedy deletion of Tannas DJ/Producer\n A...
Name: comment_text, dtype: object
116344    The article Mr. Rivado has been speedily delet...
Name: comment_text, dtype: object
3428    Birthday \n\nBorn: March 31, 1978 in New York ...
Name: comment_text, dtype: object
9680    REDIRECT Talk:James Aitken (bishop)
Name: comment_te

In [153]:
model_d2v.docvecs.most_similar('ef0345f6b5ee4573',topn=10)[1][0]

'c4d73a9da5fce32a'

In [151]:
train[train['id']=='ef0345f6b5ee4573']

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
140356,ef0345f6b5ee4573,Yeah yeah yeah... get a life faggot. 67.176.98.94,1,0,1,0,0,0
